In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline

In [2]:
import seaborn as sn
import plotly.express as px

### Request for Census Tract Level Population data from Census API

In [3]:
stateCode = ['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16',
             '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
            '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', 
             '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56']

In [4]:
len(stateCode)

51

In [5]:
import requests
import pandas as pd
from datetime import datetime
import time
import json
from io import StringIO

In [6]:
def collectAll_county(year, variables):
    basic_url = 'https://api.census.gov/data/' + year + '/acs/acs3?get=NAME,'+ variables + '&for=county:*&in=state:*&key=250f58e35283d400e4185b197e3abe9021079afe'
    r = requests.get(basic_url)
    x = r.json()
    print(x[0])
    variable_list = variables.split(',')
    column_list = ['county_name']
    for item in variable_list:
        column_list.append(item)
    column_list.append('state code')
    column_list.append('county code')
    df = pd.DataFrame(x[1:])
    df.columns = column_list
    return df

#### Total population: B01003_001E

In [7]:
year = '2011'
variables = 'B01003_001E'

In [8]:
pop_09 = collectAll_county(year,variables)
pop_09.head()

['NAME', 'B01003_001E', 'state', 'county']


,county_name,B01003_001E,state code,county code
0,"Carroll County, Maryland",167188,24,013
1,"Cecil County, Maryland",101240,24,015
2,"Charles County, Maryland",147012,24,017
3,"Dorchester County, Maryland",32587,24,019
4,"Frederick County, Maryland",233958,24,021


In [9]:
pop_09.shape

(1909, 4)

#### 1.per capita income: B19301_001E

In [10]:
variables = 'B19301_001E'
df1 = collectAll_county(year,variables)

['NAME', 'B19301_001E', 'state', 'county']


In [11]:
df1.shape

(1909, 4)

In [12]:
df1 = df1.rename(columns = {'B19301_001E': 'per_capita_income'})
df1['fips'] = df1['state code'] + df1['county code']
df1.head()

,county_name,per_capita_income,state code,county code,fips
0,"George County, Mississippi",18979,28,039,28039
1,"Grenada County, Mississippi",19513,28,043,28043
2,"Hancock County, Mississippi",22709,28,045,28045
3,"Harrison County, Mississippi",22317,28,047,28047
4,"Hinds County, Mississippi",19846,28,049,28049


#### 2.Percent of Population Below Poverty Level: (C17002_002E+C17002_003E)/C17002_001E

In [13]:
variables = 'C17002_002E,C17002_003E,C17002_001E'
df2 = collectAll_county(year,variables)

['NAME', 'C17002_002E', 'C17002_003E', 'C17002_001E', 'state', 'county']


In [14]:
df2['C17002_002E'] = df2['C17002_002E'].astype('int')
df2['C17002_003E'] = df2['C17002_003E'].astype('int')
df2['C17002_001E'] = df2['C17002_001E'].astype('int')

In [15]:
df2['percent_below_poverty'] = ((df2['C17002_002E'] + df2['C17002_003E'])/df2['C17002_001E'])*100

In [16]:
df2['fips'] = df2['state code'] + df2['county code']
df2.head()

,county_name,C17002_002E,C17002_003E,C17002_001E,state code,county code,percent_below_poverty,fips
0,"Autauga County, Alabama",2986,3479,54331,01,001,11.899284,01001
1,"Baldwin County, Alabama",8172,15273,180935,01,003,12.957692,01003
2,"Barbour County, Alabama",2811,3810,24589,01,005,26.926675,01005
3,"Bibb County, Alabama",995,1948,21979,01,007,13.390054,01007
4,"Blount County, Alabama",3477,4951,56989,01,009,14.788819,01009


In [17]:
df2.shape

(1909, 8)

#### 3.Percent of Population 25+ with Less than a 12th Grade Education: (B15002_003E,B15002_004E,B15002_005E,B15002_003E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,B15002_021E,B15002_022E,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E)/B15003_001E

In [18]:
def collectAll_county2(year, variables):
    basic_url = 'https://api.census.gov/data/' + year + '/acs/acs1?get=NAME,'+ variables + '&for=county:*&in=state:*'
    r = requests.get(basic_url)
    x = r.json()
    print(x[0])
    variable_list = variables.split(',')
    column_list = ['county_name']
    for item in variable_list:
        column_list.append(item)
    column_list.append('state code')
    column_list.append('county code')
    df = pd.DataFrame(x[1:])
    df.columns = column_list
    return df

In [19]:
variables = 'B15002_003E,B15002_004E,B15002_005E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,B15002_021E,B15002_022E,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E,B15002_001E'
df3 = collectAll_county(year,variables)

['NAME', 'B15002_003E', 'B15002_004E', 'B15002_005E', 'B15002_006E', 'B15002_007E', 'B15002_008E', 'B15002_009E', 'B15002_010E', 'B15002_020E', 'B15002_021E', 'B15002_022E', 'B15002_023E', 'B15002_024E', 'B15002_025E', 'B15002_026E', 'B15002_027E', 'B15002_001E', 'state', 'county']


In [20]:
df3.shape

(1909, 20)

In [21]:
def changeInteger(value):
    if value != None:
        return int(value)
    else:
        return np.nan

In [22]:
df3['B15002_003E'] = df3['B15002_003E'].apply(changeInteger)
df3['B15002_004E'] = df3['B15002_004E'].apply(changeInteger)
df3['B15002_005E'] = df3['B15002_005E'].apply(changeInteger)
df3['B15002_006E'] = df3['B15002_006E'].apply(changeInteger)
df3['B15002_007E'] = df3['B15002_007E'].apply(changeInteger)
df3['B15002_008E'] = df3['B15002_008E'].apply(changeInteger)
df3['B15002_009E'] = df3['B15002_009E'].apply(changeInteger)
df3['B15002_010E'] = df3['B15002_010E'].apply(changeInteger)
df3['B15002_020E'] = df3['B15002_020E'].apply(changeInteger)
df3['B15002_021E'] = df3['B15002_021E'].apply(changeInteger)
df3['B15002_022E'] = df3['B15002_022E'].apply(changeInteger)
df3['B15002_023E'] = df3['B15002_023E'].apply(changeInteger)
df3['B15002_024E'] = df3['B15002_024E'].apply(changeInteger)
df3['B15002_025E'] = df3['B15002_025E'].apply(changeInteger)
df3['B15002_026E'] = df3['B15002_026E'].apply(changeInteger)
df3['B15002_027E'] = df3['B15002_027E'].apply(changeInteger)
df3['B15002_001E'] = df3['B15002_001E'].apply(changeInteger)

In [23]:
df3['percent_no_highschool_diploma'] = ((df3['B15002_003E']+df3['B15002_004E']+df3['B15002_005E']+df3['B15002_006E']+df3['B15002_007E']+df3['B15002_008E']+df3['B15002_009E']
                                       +df3['B15002_010E']+df3['B15002_020E']+df3['B15002_021E']+df3['B15002_022E']+df3['B15002_023E']+df3['B15002_024E']+df3['B15002_025E']+df3['B15002_003E']
                                       +df3['B15002_026E']+df3['B15002_027E'])/df3['B15002_001E'])*100

In [24]:
df3['fips'] = df3['state code'] + df3['county code']
df3.head()

,county_name,B15002_003E,B15002_004E,B15002_005E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,...,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E,B15002_001E,state code,county code,percent_no_highschool_diploma,fips
0,"Talladega County, Alabama",296.0,215.0,397.0,1374.0,785.0,1366.0,1369.0,658.0,117.0,...,1276.0,1439.0,1254.0,1296.0,461.0,55893.0,01,121,23.409014,01121
1,"Tallapoosa County, Alabama",195.0,91.0,317.0,461.0,339.0,493.0,927.0,473.0,203.0,...,605.0,530.0,465.0,640.0,255.0,28973.0,01,123,22.172367,01123
2,"Tuscaloosa County, Alabama",738.0,287.0,401.0,1372.0,1118.0,2149.0,1694.0,846.0,419.0,...,741.0,803.0,1542.0,1919.0,953.0,114653.0,01,125,14.305775,01125
3,"Walker County, Alabama",219.0,200.0,584.0,912.0,1109.0,1574.0,919.0,498.0,253.0,...,869.0,1264.0,1001.0,1104.0,348.0,46592.0,01,127,24.927026,01127
4,"Winston County, Alabama",181.0,207.0,244.0,364.0,270.0,588.0,360.0,240.0,195.0,...,432.0,425.0,502.0,425.0,244.0,17074.0,01,133,29.184725,01133


#### 4.Percent of Population Living in Mobile Homes: (B25033_006E+B25033_007E+B25033_012E+B25033_013E)/B25033_001E

In [25]:
variables = 'B25033_006E,B25033_007E,B25033_012E,B25033_013E,B25033_001E'
df4 = collectAll_county(year,variables)

['NAME', 'B25033_006E', 'B25033_007E', 'B25033_012E', 'B25033_013E', 'B25033_001E', 'state', 'county']


In [26]:
df4.shape

(1909, 8)

In [27]:
df4['B25033_006E'] = df4['B25033_006E'].apply(changeInteger)
df4['B25033_007E'] = df4['B25033_007E'].apply(changeInteger)
df4['B25033_012E'] = df4['B25033_012E'].apply(changeInteger)
df4['B25033_013E'] = df4['B25033_013E'].apply(changeInteger)
df4['B25033_001E'] = df4['B25033_001E'].apply(changeInteger)

In [28]:
df4['percent_mobile_homes'] = ((df4['B25033_006E'] + df4['B25033_007E'] + df4['B25033_012E'] + df4['B25033_013E'])/df4['B25033_001E'])*100

In [29]:
df4['fips'] = df4['state code'] + df4['county code']
df4.head()

,county_name,B25033_006E,B25033_007E,B25033_012E,B25033_013E,B25033_001E,state code,county code,percent_mobile_homes,fips
0,"Bonner County, Idaho",6017.0,199.0,1929.0,34.0,40462.0,16,017,20.214028,16017
1,"Bonneville County, Idaho",3916.0,0.0,1303.0,0.0,103514.0,16,019,5.041830,16019
2,"Canyon County, Idaho",10496.0,107.0,4797.0,0.0,186664.0,16,027,8.250118,16027
3,"Cassia County, Idaho",2221.0,18.0,1488.0,0.0,22524.0,16,031,16.546795,16031
4,"Elmore County, Idaho",2275.0,35.0,402.0,0.0,25993.0,16,039,10.433578,16039


#### 5.Percent of Population with No Vehicle Available: (B25044_003E+B25044_010E)/B25044_001E

In [30]:
variables = 'B25044_003E,B25044_010E,B25044_001E'
df5 = collectAll_county(year,variables)

['NAME', 'B25044_003E', 'B25044_010E', 'B25044_001E', 'state', 'county']


In [31]:
df5.shape

(1909, 6)

In [32]:
df5['B25044_003E'] = df5['B25044_003E'].apply(changeInteger)
df5['B25044_010E'] = df5['B25044_010E'].apply(changeInteger)
df5['B25044_001E'] = df5['B25044_001E'].apply(changeInteger)

In [33]:
df5['percent_no_vehicle'] = ((df5['B25044_003E'] + df5['B25044_010E'])/df5['B25044_001E'])*100

In [34]:
df5['fips'] = df5['state code'] + df5['county code']
df5.head()

,county_name,B25044_003E,B25044_010E,B25044_001E,state code,county code,percent_no_vehicle,fips
0,"Lake County, Florida",2341,2967,116576,12,069,4.553253,12069
1,"Lee County, Florida",4091,7361,237009,12,071,4.831884,12071
2,"Leon County, Florida",1459,6076,109822,12,073,6.861103,12073
3,"Levy County, Florida",463,716,16466,12,075,7.160209,12075
4,"Manatee County, Florida",2495,3935,130834,12,081,4.914625,12081


#### 6.Percent of Population Unemployed:sum(B21005_006E, B21005_007E, B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E)/B21005_001E

In [35]:
variables = 'B21005_006E,B21005_007E,B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E,B21005_001E'
df6 = collectAll_county2(year,variables)

['NAME', 'B21005_006E', 'B21005_007E', 'B21005_011E', 'B21005_012E', 'B21005_017E', 'B21005_022E', 'B21005_028E', 'B21005_029E', 'B21005_033E', 'B21005_034E', 'B21005_001E', 'state', 'county']


In [36]:
df6.shape

(822, 14)

In [37]:
df6['B21005_006E'] = df6['B21005_006E'].apply(changeInteger)
df6['B21005_007E'] = df6['B21005_007E'].apply(changeInteger)
df6['B21005_011E'] = df6['B21005_011E'].apply(changeInteger)
df6['B21005_012E'] = df6['B21005_012E'].apply(changeInteger)
df6['B21005_017E'] = df6['B21005_017E'].apply(changeInteger)
df6['B21005_022E'] = df6['B21005_022E'].apply(changeInteger)
df6['B21005_028E'] = df6['B21005_028E'].apply(changeInteger)
df6['B21005_029E'] = df6['B21005_029E'].apply(changeInteger)
df6['B21005_033E'] = df6['B21005_033E'].apply(changeInteger)
df6['B21005_034E'] = df6['B21005_034E'].apply(changeInteger)
df6['B21005_001E'] = df6['B21005_001E'].apply(changeInteger)

In [38]:
df6['percent_unemployed_CDC'] = ((df6['B21005_006E']+df6['B21005_007E']+df6['B21005_011E']+df6['B21005_012E']+df6['B21005_017E']+df6['B21005_022E']+df6['B21005_028E']+df6['B21005_029E']+df6['B21005_033E']+df6['B21005_034E'])/df6['B21005_001E'])*100

In [39]:
df6['fips'] = df6['state code'] + df6['county code']
df6.head()

,county_name,B21005_006E,B21005_007E,B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E,B21005_001E,state code,county code,percent_unemployed_CDC,fips
0,"Sumter County, Florida",0,112,518,4932,0,658,507,2599,204,10312,44694,12,119,44.395221,12119
1,"Volusia County, Florida",268,1107,10662,33198,589,8206,1148,7464,3578,22796,296352,12,127,30.037253,12127
2,"Barrow County, Georgia",0,0,3792,2272,23,1486,22,226,27,2485,43514,13,013,23.746380,13013
3,"Bartow County, Georgia",0,76,2491,5729,308,2515,241,796,714,4136,62986,13,015,26.999651,13015
4,"Bibb County, Georgia",222,166,5050,11004,471,2564,0,1560,480,7655,95941,13,021,30.406187,13021


#### 7.Percent of Population Living In Accommodations with Less Than 1 Room Per Person/Crowding: (B25014_005E+B25014_006E+B25014_007E+B25014_011E+B25014_012E+B25014_013E)/B25014_001E

In [40]:
variables = 'B25014_005E,B25014_006E,B25014_007E,B25014_011E,B25014_012E,B25014_013E,B25014_001E'
df7 = collectAll_county(year,variables)

['NAME', 'B25014_005E', 'B25014_006E', 'B25014_007E', 'B25014_011E', 'B25014_012E', 'B25014_013E', 'B25014_001E', 'state', 'county']


In [41]:
df7.shape

(1909, 10)

In [42]:
df7['B25014_005E'] = df7['B25014_005E'].apply(changeInteger)
df7['B25014_006E'] = df7['B25014_006E'].apply(changeInteger)
df7['B25014_007E'] = df7['B25014_007E'].apply(changeInteger)
df7['B25014_011E'] = df7['B25014_011E'].apply(changeInteger)
df7['B25014_012E'] = df7['B25014_012E'].apply(changeInteger)
df7['B25014_013E'] = df7['B25014_013E'].apply(changeInteger)
df7['B25014_001E'] = df7['B25014_001E'].apply(changeInteger)

In [43]:
df7['percent_overcrowding'] = ((df7['B25014_005E']+df7['B25014_006E']+df7['B25014_007E']+df7['B25014_011E']+df7['B25014_012E']+df7['B25014_013E'])/df7['B25014_001E'])*100

In [44]:
df7['fips'] = df7['state code'] + df7['county code']
df7.head()

,county_name,B25014_005E,B25014_006E,B25014_007E,B25014_011E,B25014_012E,B25014_013E,B25014_001E,state code,county code,percent_overcrowding,fips
0,"Madison County, Indiana",259.0,9.0,0.0,362.0,60.0,32.0,50290.0,18,095,1.435673,18095
1,"Marion County, Indiana",1942.0,184.0,59.0,3888.0,1029.0,521.0,356059.0,18,097,2.140937,18097
2,"Marshall County, Indiana",215.0,0.0,0.0,69.0,29.0,13.0,17662.0,18,099,1.845771,18099
3,"Miami County, Indiana",73.0,0.0,0.0,146.0,0.0,0.0,13313.0,18,103,1.645009,18103
4,"Monroe County, Indiana",158.0,37.0,0.0,271.0,57.0,158.0,53659.0,18,105,1.269125,18105


#### 8.Percent of Housing Units with 10+ Units in Structure: (B25024_007E+B25024_008E+B25024_009E)/B25024_001E

In [45]:
variables = 'B25024_007E,B25024_008E,B25024_009E,B25024_001E'
df8 = collectAll_county(year,variables)

['NAME', 'B25024_007E', 'B25024_008E', 'B25024_009E', 'B25024_001E', 'state', 'county']


In [46]:
df8.shape

(1909, 7)

In [47]:
df8['B25024_007E'] = df8['B25024_007E'].apply(changeInteger)
df8['B25024_008E'] = df8['B25024_008E'].apply(changeInteger)
df8['B25024_009E'] = df8['B25024_009E'].apply(changeInteger)
df8['B25024_001E'] = df8['B25024_001E'].apply(changeInteger)

In [48]:
df8['percent_multi_unit_housing'] = ((df8['B25024_007E']+df8['B25024_008E']+df8['B25024_009E'])/df8['B25024_001E'])*100

In [49]:
df8['fips'] = df8['state code'] + df8['county code']
df8.head()

,county_name,B25024_007E,B25024_008E,B25024_009E,B25024_001E,state code,county code,percent_multi_unit_housing,fips
0,"Greenup County, Kentucky",128.0,77.0,151.0,16328.0,21,089,2.180304,21089
1,"Hardin County, Kentucky",669.0,180.0,250.0,43330.0,21,093,2.536349,21093
2,"Harlan County, Kentucky",242.0,94.0,62.0,13469.0,21,095,2.954934,21095
3,"Henderson County, Kentucky",446.0,94.0,234.0,20326.0,21,101,3.807931,21101
4,"Hopkins County, Kentucky",258.0,162.0,0.0,21165.0,21,107,1.984408,21107


#### 9.Percent Of Population 65+: (B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_049E)/B01001_001E

In [50]:
variables = 'B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B01001_001E'
df9 = collectAll_county(year,variables)

['NAME', 'B01001_020E', 'B01001_021E', 'B01001_022E', 'B01001_023E', 'B01001_024E', 'B01001_025E', 'B01001_044E', 'B01001_045E', 'B01001_046E', 'B01001_047E', 'B01001_048E', 'B01001_049E', 'B01001_001E', 'state', 'county']


In [51]:
df9.shape

(1909, 16)

In [52]:
df9['B01001_020E'] = df9['B01001_020E'].apply(changeInteger)
df9['B01001_021E'] = df9['B01001_021E'].apply(changeInteger)
df9['B01001_022E'] = df9['B01001_022E'].apply(changeInteger)
df9['B01001_023E'] = df9['B01001_023E'].apply(changeInteger)
df9['B01001_024E'] = df9['B01001_024E'].apply(changeInteger)
df9['B01001_025E'] = df9['B01001_025E'].apply(changeInteger)
df9['B01001_044E'] = df9['B01001_044E'].apply(changeInteger)
df9['B01001_045E'] = df9['B01001_045E'].apply(changeInteger)
df9['B01001_046E'] = df9['B01001_046E'].apply(changeInteger)
df9['B01001_047E'] = df9['B01001_047E'].apply(changeInteger)
df9['B01001_048E'] = df9['B01001_048E'].apply(changeInteger)
df9['B01001_049E'] = df9['B01001_049E'].apply(changeInteger)
df9['B01001_001E'] = df9['B01001_001E'].apply(changeInteger)

In [53]:
df9['percent_age_65_and_older'] = ((df9['B01001_020E']+df9['B01001_021E']+df9['B01001_022E']+df9['B01001_023E']+df9['B01001_024E']+df9['B01001_025E']+df9['B01001_044E']
                                  +df9['B01001_045E']+df9['B01001_046E']+df9['B01001_047E']+df9['B01001_048E']+df9['B01001_049E'])/df9['B01001_001E'])*100

In [54]:
df9['fips'] = df9['state code'] + df9['county code']
df9.head()

,county_name,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B01001_001E,state code,county code,percent_age_65_and_older,fips
0,"LaSalle County, Illinois",1093,1385,1890,1687,1089,888,1074,1474,2395,1787,1875,2120,113787,17,099,16.484308,17099
1,"Lee County, Illinois",436,335,663,533,299,282,160,497,796,551,409,650,35866,17,103,15.644343,17103
2,"Livingston County, Illinois",260,421,672,462,453,245,354,378,896,629,564,714,38940,17,105,15.531587,17105
3,"Logan County, Illinois",313,299,467,409,183,313,223,331,720,465,503,538,30251,17,107,15.748240,17107
4,"McDonough County, Illinois",195,303,552,321,374,154,249,290,595,702,443,451,32602,17,109,14.198515,17109


#### 10.Percent Of Population Who Do Not Speak English: B99163_005E/B99163_001E

In [55]:
variables = 'B99163_005E,B99163_001E'
df10 = collectAll_county(year,variables)

['NAME', 'B99163_005E', 'B99163_001E', 'state', 'county']


In [56]:
df10.shape

(1909, 5)

In [57]:
df10['B99163_005E'] = df10['B99163_005E'].apply(changeInteger)
df10['B99163_001E'] = df10['B99163_001E'].apply(changeInteger)

In [58]:
df10['percent_limited_english_abilities'] = (df10['B99163_005E']/df10['B99163_001E'])*100

In [59]:
df10['fips'] = df10['state code'] + df10['county code']
df10.head()

,county_name,B99163_005E,B99163_001E,state code,county code,percent_limited_english_abilities,fips
0,"Chatham County, Georgia",19492,247846,13,051,7.864561,13051
1,"Chattooga County, Georgia",901,24320,13,055,3.704770,13055
2,"Cherokee County, Georgia",23809,199444,13,057,11.937687,13057
3,"Clarke County, Georgia",14038,110008,13,059,12.760890,13059
4,"Clayton County, Georgia",46306,238515,13,063,19.414293,13063


#### 11.Percent of Population 17 Years of Age and Under: (B01001_003E+B01001_004E+B01001_005E+B01001_006E+B01001_027E+B01001_028E+B01001_029E+B01001_030E)/B01003_001E

In [60]:
variables = 'B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_027E,B01001_028E,B01001_029E,B01001_030E,B01003_001E'
df11 = collectAll_county(year,variables)

['NAME', 'B01001_003E', 'B01001_004E', 'B01001_005E', 'B01001_006E', 'B01001_027E', 'B01001_028E', 'B01001_029E', 'B01001_030E', 'B01003_001E', 'state', 'county']


In [61]:
df11.shape

(1909, 12)

In [62]:
df11['B01001_003E'] = df11['B01001_003E'].apply(changeInteger)
df11['B01001_004E'] = df11['B01001_004E'].apply(changeInteger)
df11['B01001_005E'] = df11['B01001_005E'].apply(changeInteger)
df11['B01001_006E'] = df11['B01001_006E'].apply(changeInteger)
df11['B01001_027E'] = df11['B01001_027E'].apply(changeInteger)
df11['B01001_028E'] = df11['B01001_028E'].apply(changeInteger)
df11['B01001_029E'] = df11['B01001_029E'].apply(changeInteger)
df11['B01001_030E'] = df11['B01001_030E'].apply(changeInteger)
df11['B01003_001E'] = df11['B01003_001E'].apply(changeInteger)

In [63]:
df11['percent_age_17_and_younger'] = ((df11['B01001_003E']+df11['B01001_004E']+df11['B01001_005E']+df11['B01001_006E']+df11['B01001_027E']+df11['B01001_028E']+df11['B01001_029E']+df11['B01001_030E'])/df11['B01003_001E'])*100

In [64]:
df11['fips'] = df11['state code'] + df11['county code']
df11.head()

,county_name,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_027E,B01001_028E,B01001_029E,B01001_030E,B01003_001E,state code,county code,percent_age_17_and_younger,fips
0,"Carroll County, Maryland",4689,5198,6924,4043,4340,5773,5948,4065,167188,24,013,24.511329,24013
1,"Cecil County, Maryland",2999,3616,3668,2473,3377,3307,3680,2189,101240,24,015,24.999012,24015
2,"Charles County, Maryland",4906,5490,5688,3801,4466,4999,5609,3651,147012,24,017,26.263162,24017
3,"Dorchester County, Maryland",1070,792,1103,576,947,788,1001,718,32587,24,019,21.465615,24019
4,"Frederick County, Maryland",7637,8418,8752,5526,7192,7760,8632,5008,233958,24,021,25.186145,24021


#### 12.Percent Minority: 1-(B03002_003E/B03002_001E)

In [65]:
variables = 'B03002_003E,B03002_001E'
df12 = collectAll_county(year,variables)

['NAME', 'B03002_003E', 'B03002_001E', 'state', 'county']


In [66]:
df12.shape

(1909, 5)

In [67]:
df12['B03002_003E'] = df12['B03002_003E'].apply(changeInteger)
df12['B03002_001E'] = df12['B03002_001E'].apply(changeInteger)

In [68]:
df12['percent_minorities'] = (1 - (df12['B03002_003E']/df12['B03002_001E']))*100

In [69]:
df12['fips'] = df12['state code'] + df12['county code']
df12.head()

,county_name,B03002_003E,B03002_001E,state code,county code,percent_minorities,fips
0,"Calhoun County, Alabama",86920.0,118195.0,01,015,26.460510,01015
1,"Coffee County, Alabama",36152.0,50044.0,01,031,27.759572,01031
2,"Elmore County, Alabama",59372.0,79461.0,01,051,25.281585,01051
3,"Houston County, Alabama",69657.0,101546.0,01,069,31.403502,01069
4,"Jefferson County, Alabama",339970.0,658586.0,01,073,48.378799,01073


#### 13.Percent of Population with a Disability: (B18101_004E+B18101_007E+B18101_010E+B18101_013E+B18101_016E+B18101_019E)/B18101_002E + (B18101_023E+B18101_026E+B18101_029E+B18101_032E+B18101_035E+B18101_038E)/B18101_021E

In [70]:
variables = 'B18101_004E,B18101_007E,B18101_010E,B18101_013E,B18101_016E,B18101_019E,B18101_002E,B18101_023E,B18101_026E,B18101_029E,B18101_032E,B18101_035E,B18101_038E,B18101_021E'
df13 = collectAll_county2(year,variables)

['NAME', 'B18101_004E', 'B18101_007E', 'B18101_010E', 'B18101_013E', 'B18101_016E', 'B18101_019E', 'B18101_002E', 'B18101_023E', 'B18101_026E', 'B18101_029E', 'B18101_032E', 'B18101_035E', 'B18101_038E', 'B18101_021E', 'state', 'county']


In [71]:
df13.shape

(822, 17)

In [72]:
df13['B18101_004E'] = df13['B18101_004E'].apply(changeInteger)
df13['B18101_007E'] = df13['B18101_007E'].apply(changeInteger)
df13['B18101_010E'] = df13['B18101_010E'].apply(changeInteger)
df13['B18101_013E'] = df13['B18101_013E'].apply(changeInteger)
df13['B18101_016E'] = df13['B18101_016E'].apply(changeInteger)
df13['B18101_002E'] = df13['B18101_002E'].apply(changeInteger)
df13['B18101_023E'] = df13['B18101_023E'].apply(changeInteger)
df13['B18101_026E'] = df13['B18101_026E'].apply(changeInteger)
df13['B18101_029E'] = df13['B18101_029E'].apply(changeInteger)
df13['B18101_032E'] = df13['B18101_032E'].apply(changeInteger)
df13['B18101_035E'] = df13['B18101_035E'].apply(changeInteger)
df13['B18101_038E'] = df13['B18101_038E'].apply(changeInteger)
df13['B18101_021E'] = df13['B18101_021E'].apply(changeInteger)

In [73]:
df13['percent_disabled'] = ((df13['B18101_004E']+df13['B18101_007E']+df13['B18101_010E']+df13['B18101_013E']+df13['B18101_016E'])/df13['B18101_002E'] + (df13['B18101_023E']+df13['B18101_026E']+df13['B18101_029E']+df13['B18101_032E']+df13['B18101_035E']+df13['B18101_038E'])/df13['B18101_021E'])*100

In [74]:
df13['fips'] = df13['state code'] + df13['county code']
df13.head()

,county_name,B18101_004E,B18101_007E,B18101_010E,B18101_013E,B18101_016E,B18101_019E,B18101_002E,B18101_023E,B18101_026E,B18101_029E,B18101_032E,B18101_035E,B18101_038E,B18101_021E,state code,county code,percent_disabled,fips
0,"Sumter County, Florida",0,145,260,985,4202,3905,43364,0,198,82,2798,1075,2657,46051,12,119,27.683442,12119
1,"Volusia County, Florida",115,2816,5066,15785,6211,10077,239174,276,1666,2132,15274,5635,14445,249670,12,127,28.332288,12127
2,"Barrow County, Georgia",0,0,620,1568,387,409,35591,0,456,480,2570,555,944,33704,13,013,22.084845,13013
3,"Bartow County, Georgia",0,216,64,2611,1209,924,48361,0,92,616,3579,1656,1140,51103,13,015,22.338149,13015
4,"Bibb County, Georgia",0,1464,939,3340,1277,1555,70252,174,1656,906,5245,2256,2808,82052,13,021,25.891053,13021


#### 14.Percent of Population Living in Group Quarters: B26001_001E/B01003_001E

In [75]:
variables = 'B26001_001E,B01003_001E'
df14 = collectAll_county(year,variables)

['NAME', 'B26001_001E', 'B01003_001E', 'state', 'county']


In [76]:
df14.shape

(1909, 5)

In [77]:
df14['B26001_001E'] = df14['B26001_001E'].apply(changeInteger)
df14['B01003_001E'] = df14['B01003_001E'].apply(changeInteger)

In [78]:
df14['percent_institutionalized_in_group_quarters'] = (df14['B26001_001E']/df14['B01003_001E'])*100

In [79]:
df14['fips'] = df14['state code'] + df14['county code']
df14.head()

,county_name,B26001_001E,B01003_001E,state code,county code,percent_institutionalized_in_group_quarters,fips
0,"Carroll County, Maryland",4062.0,167188,24,013,2.429600,24013
1,"Cecil County, Maryland",1835.0,101240,24,015,1.812525,24015
2,"Charles County, Maryland",1413.0,147012,24,017,0.961146,24017
3,"Dorchester County, Maryland",625.0,32587,24,019,1.917943,24019
4,"Frederick County, Maryland",4550.0,233958,24,021,1.944794,24021


#### 15.Percent of Children Living in Single Parent Households: (B09008_010E+B09008_011E+B09008_012E)/B09008_001E

In [80]:
variables = 'B09008_010E,B09008_011E,B09008_012E,B09008_001E'
df15 = collectAll_county2(year,variables)

['NAME', 'B09008_010E', 'B09008_011E', 'B09008_012E', 'B09008_001E', 'state', 'county']


In [81]:
df15.shape

(822, 7)

In [82]:
df15['B09008_010E'] = df15['B09008_010E'].apply(changeInteger)
df15['B09008_011E'] = df15['B09008_011E'].apply(changeInteger)
df15['B09008_012E'] = df15['B09008_012E'].apply(changeInteger)
df15['B09008_001E'] = df15['B09008_001E'].apply(changeInteger)

In [83]:
df15['percent_single_parent_households_CDC'] = ((df15['B09008_010E'] + df15['B09008_011E']+df15['B09008_012E'])/df15['B09008_001E'])*100

In [84]:
df15['fips'] = df15['state code'] + df15['county code']
df15.head()

,county_name,B09008_010E,B09008_011E,B09008_012E,B09008_001E,state code,county code,percent_single_parent_households_CDC,fips
0,"Cleveland County, North Carolina",1094.0,7623.0,0.0,22407.0,37,045,38.903021,37045
1,"Craven County, North Carolina",835.0,6774.0,0.0,23908.0,37,049,31.826167,37049
2,"Cumberland County, North Carolina",3814.0,28896.0,0.0,85801.0,37,051,38.123099,37051
3,"Davidson County, North Carolina",821.0,9350.0,101.0,38019.0,37,057,27.018070,37057
4,"Durham County, North Carolina",2541.0,20092.0,168.0,61683.0,37,063,36.964804,37063


### age 18 to 34
#### (B01001_007E+B01001_008E+B01001_009E+B01001_010E+B01001_011E+B01001_012E+B01001_031E+B01001_032E+B01001_033E+B01001_034E+B01001_035E+B01001_036E)/B01001_001E

In [85]:
variables = 'B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_001E'
df16 = collectAll_county(year,variables)

['NAME', 'B01001_007E', 'B01001_008E', 'B01001_009E', 'B01001_010E', 'B01001_011E', 'B01001_012E', 'B01001_031E', 'B01001_032E', 'B01001_033E', 'B01001_034E', 'B01001_035E', 'B01001_036E', 'B01001_001E', 'state', 'county']


In [86]:
df16['B01001_007E'] = df16['B01001_007E'].apply(changeInteger)
df16['B01001_008E'] = df16['B01001_008E'].apply(changeInteger)
df16['B01001_009E'] = df16['B01001_009E'].apply(changeInteger)
df16['B01001_010E'] = df16['B01001_010E'].apply(changeInteger)
df16['B01001_011E'] = df16['B01001_011E'].apply(changeInteger)
df16['B01001_012E'] = df16['B01001_012E'].apply(changeInteger)
df16['B01001_031E'] = df16['B01001_031E'].apply(changeInteger)
df16['B01001_032E'] = df16['B01001_032E'].apply(changeInteger)
df16['B01001_033E'] = df16['B01001_033E'].apply(changeInteger)
df16['B01001_034E'] = df16['B01001_034E'].apply(changeInteger)
df16['B01001_035E'] = df16['B01001_035E'].apply(changeInteger)
df16['B01001_036E'] = df16['B01001_036E'].apply(changeInteger)
df16['B01001_001E'] = df16['B01001_001E'].apply(changeInteger)

In [87]:
df16['percent_age_18_to_34'] = ((df16['B01001_007E'] + df16['B01001_008E'] +df16['B01001_009E'] +df16['B01001_010E'] +df16['B01001_011E']+df16['B01001_012E']+df16['B01001_031E']+df16['B01001_032E']+df16['B01001_033E']+df16['B01001_034E']+df16['B01001_035E']+df16['B01001_036E'])/df16['B01001_001E'])*100

In [88]:
df16['fips'] = df16['state code'] + df16['county code']
df16.head()

,county_name,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_001E,state code,county code,percent_age_18_to_34,fips
0,"LaSalle County, Illinois",1639,734,942,1897,3633,3187,1521,911,706,1257,3242,3096,113787,17,099,20.006679,17099
1,"Lee County, Illinois",504,317,289,712,1109,1118,496,245,34,596,896,933,35866,17,103,20.211342,17103
2,"Livingston County, Illinois",477,166,127,966,1111,1413,420,205,203,631,1107,1355,38940,17,105,21.009245,17105
3,"Logan County, Illinois",598,300,198,494,1459,1257,462,85,207,649,923,1024,30251,17,107,25.308254,17107
4,"McDonough County, Illinois",2186,794,716,1507,1076,726,1486,709,1039,1421,787,705,32602,17,109,40.341083,17109


In [89]:
df16.shape

(1909, 18)

### Age 35 to 44
#### (B01001_013E+B01001_014E+B01001_037E+B01001_038E)/B01001_001E

In [90]:
variables = 'B01001_013E,B01001_014E,B01001_037E,B01001_038E,B01001_001E'
df17 = collectAll_county(year,variables)

['NAME', 'B01001_013E', 'B01001_014E', 'B01001_037E', 'B01001_038E', 'B01001_001E', 'state', 'county']


In [91]:
df17['B01001_013E'] = df17['B01001_013E'].apply(changeInteger)
df17['B01001_014E'] = df17['B01001_014E'].apply(changeInteger)
df17['B01001_037E'] = df17['B01001_037E'].apply(changeInteger)
df17['B01001_038E'] = df17['B01001_038E'].apply(changeInteger)
df17['B01001_001E'] = df17['B01001_001E'].apply(changeInteger)

In [92]:
df17['percent_age_35_to_44'] = ((df17['B01001_013E']+df17['B01001_014E'] +df17['B01001_037E']+df17['B01001_038E'])/df17['B01001_001E'])*100

In [93]:
df17['fips'] = df17['state code'] + df17['county code']
df17.head()

,county_name,B01001_013E,B01001_014E,B01001_037E,B01001_038E,B01001_001E,state code,county code,percent_age_35_to_44,fips
0,"LaSalle County, Illinois",3357,3826,2969,3754,113787,17,099,12.221080,17099
1,"Lee County, Illinois",1322,1251,1104,1143,35866,17,103,13.438912,17103
2,"Livingston County, Illinois",1522,1153,1122,1358,38940,17,105,13.238315,17105
3,"Logan County, Illinois",1156,794,710,1174,30251,17,107,12.673961,17107
4,"McDonough County, Illinois",751,574,732,732,32602,17,109,8.554690,17109


In [94]:
df17.shape

(1909, 10)

### age 45 to 54
#### (B01001_015E+B01001_016E+B01001_039E+B01001_040E)/B01001_001E

In [95]:
variables = 'B01001_015E,B01001_016E,B01001_039E,B01001_040E,B01001_001E'
df18 = collectAll_county(year,variables)

['NAME', 'B01001_015E', 'B01001_016E', 'B01001_039E', 'B01001_040E', 'B01001_001E', 'state', 'county']


In [96]:
df18['B01001_015E'] = df18['B01001_015E'].apply(changeInteger)
df18['B01001_016E'] = df18['B01001_016E'].apply(changeInteger)
df18['B01001_039E'] = df18['B01001_039E'].apply(changeInteger)
df18['B01001_040E'] = df18['B01001_040E'].apply(changeInteger)
df18['B01001_001E'] = df18['B01001_001E'].apply(changeInteger)

In [97]:
df18['percent_age_45_to_54'] = ((df18['B01001_015E']+df18['B01001_016E'] +df18['B01001_039E']+df18['B01001_040E'])/df18['B01001_001E'])*100

In [98]:
df18['fips'] = df18['state code'] + df18['county code']
df18.head()

,county_name,B01001_015E,B01001_016E,B01001_039E,B01001_040E,B01001_001E,state code,county code,percent_age_45_to_54,fips
0,"LaSalle County, Illinois",4472,4582,4271,4568,113787,17,099,15.724995,17099
1,"Lee County, Illinois",1662,1475,1329,1345,35866,17,103,16.201974,17103
2,"Livingston County, Illinois",1671,1589,1439,1559,38940,17,105,16.070878,17105
3,"Logan County, Illinois",1150,1152,1053,1112,30251,17,107,14.766454,17107
4,"McDonough County, Illinois",777,931,811,994,32602,17,109,10.775413,17109


In [99]:
df18.shape

(1909, 10)

### age 55 to 64
#### (B01001_017E+B01001_018E+B01001_019E+B01001_041E+B01001_042E+B01001_043E)/B01001_001E

In [100]:
variables = 'B01001_017E,B01001_018E,B01001_019E,B01001_041E,B01001_042E,B01001_043E,B01001_001E'
df19 = collectAll_county(year,variables)

['NAME', 'B01001_017E', 'B01001_018E', 'B01001_019E', 'B01001_041E', 'B01001_042E', 'B01001_043E', 'B01001_001E', 'state', 'county']


In [101]:
df19['B01001_017E'] = df19['B01001_017E'].apply(changeInteger)
df19['B01001_018E'] = df19['B01001_018E'].apply(changeInteger)
df19['B01001_019E'] = df19['B01001_019E'].apply(changeInteger)
df19['B01001_041E'] = df19['B01001_041E'].apply(changeInteger)
df19['B01001_042E'] = df19['B01001_042E'].apply(changeInteger)
df19['B01001_043E'] = df19['B01001_043E'].apply(changeInteger)
df19['B01001_001E'] = df19['B01001_001E'].apply(changeInteger)

In [102]:
df19['percent_age_55_to_64'] = ((df19['B01001_017E']+df19['B01001_018E'] +df19['B01001_019E']+df19['B01001_041E']+df19['B01001_042E']+df19['B01001_043E'])/df19['B01001_001E'])*100

In [103]:
df19['fips'] = df19['state code'] + df19['county code']
df19.head()

,county_name,B01001_017E,B01001_018E,B01001_019E,B01001_041E,B01001_042E,B01001_043E,B01001_001E,state code,county code,percent_age_55_to_64,fips
0,"LaSalle County, Illinois",3827,1508,1893,3930,1512,1692,113787,17,099,12.621829,17099
1,"Lee County, Illinois",1165,488,724,1128,581,571,35866,17,103,12.984442,17103
2,"Livingston County, Illinois",1458,495,520,1149,474,746,38940,17,105,12.434515,17105
3,"Logan County, Illinois",992,380,421,948,382,515,30251,17,107,12.026049,17107
4,"McDonough County, Illinois",868,282,598,1024,298,406,32602,17,109,10.661923,17109


In [104]:
df19.shape

(1909, 12)

### Combine 19 dataframes together

In [105]:
df = df1[['fips','per_capita_income']].merge(df2[['fips', 'percent_below_poverty']], on = 'fips', how = 'left')

In [106]:
df = df.merge(df3[['fips', 'percent_no_highschool_diploma']], on = 'fips', how = 'left')
df = df.merge(df4[['fips', 'percent_mobile_homes']], on = 'fips', how = 'left')
df = df.merge(df5[['fips', 'percent_no_vehicle']], on = 'fips', how = 'left')
df = df.merge(df7[['fips', 'percent_overcrowding']], on = 'fips', how = 'left')
df = df.merge(df8[['fips', 'percent_multi_unit_housing']], on = 'fips', how = 'left')
df = df.merge(df9[['fips', 'percent_age_65_and_older']], on = 'fips', how = 'left')
df = df.merge(df10[['fips', 'percent_limited_english_abilities']], on = 'fips', how = 'left')
df = df.merge(df12[['fips', 'percent_minorities']], on = 'fips', how = 'left')
df = df.merge(df14[['fips', 'percent_institutionalized_in_group_quarters']], on = 'fips', how = 'left')
df = df.merge(df16[['fips', 'percent_age_18_to_34']], on = 'fips', how = 'left')
df = df.merge(df17[['fips', 'percent_age_35_to_44']], on = 'fips', how = 'left')
df = df.merge(df18[['fips', 'percent_age_45_to_54']], on = 'fips', how = 'left')
df = df.merge(df19[['fips', 'percent_age_55_to_64']], on = 'fips', how = 'left')
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_minorities,percent_institutionalized_in_group_quarters,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64
0,28039,18979,15.922081,15.728385,24.213965,5.849931,3.519980,NaN,13.132296,2.234717,NaN,2.542448,22.568093,12.637071,13.715953,11.160241
1,28043,19513,20.664275,20.825490,15.856503,7.677703,NaN,3.298868,15.107881,0.602115,NaN,2.011337,19.477967,14.138782,13.663375,12.931980
2,28045,22709,21.007065,15.269759,13.616123,3.630252,NaN,2.381277,15.646923,4.240035,NaN,1.092015,18.557450,12.493473,15.510704,14.171226
3,28047,22317,18.766355,15.908550,10.935382,5.047220,3.580701,8.753282,11.791717,6.553957,32.978072,2.669512,24.974859,12.712239,14.559666,11.466076
4,28049,19846,23.845571,16.424755,4.613637,7.772039,2.698577,7.465936,10.883400,3.125699,71.868803,3.390065,26.026529,12.004210,13.696804,10.982152


In [107]:
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_minorities,percent_institutionalized_in_group_quarters,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64
0,28039,18979,15.922081,15.728385,24.213965,5.849931,3.519980,NaN,13.132296,2.234717,NaN,2.542448,22.568093,12.637071,13.715953,11.160241
1,28043,19513,20.664275,20.825490,15.856503,7.677703,NaN,3.298868,15.107881,0.602115,NaN,2.011337,19.477967,14.138782,13.663375,12.931980
2,28045,22709,21.007065,15.269759,13.616123,3.630252,NaN,2.381277,15.646923,4.240035,NaN,1.092015,18.557450,12.493473,15.510704,14.171226
3,28047,22317,18.766355,15.908550,10.935382,5.047220,3.580701,8.753282,11.791717,6.553957,32.978072,2.669512,24.974859,12.712239,14.559666,11.466076
4,28049,19846,23.845571,16.424755,4.613637,7.772039,2.698577,7.465936,10.883400,3.125699,71.868803,3.390065,26.026529,12.004210,13.696804,10.982152


In [108]:
df = df.sort_values('fips')
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_minorities,percent_institutionalized_in_group_quarters,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64
1010,01001,24330,11.899284,15.417352,20.549005,5.046126,2.575994,2.098437,11.995245,2.651026,NaN,0.731529,20.747988,14.743965,14.970739,10.771763
1011,01003,26848,12.957692,12.145905,13.050596,3.371146,2.076904,17.290621,16.961999,5.566193,NaN,1.268519,19.044707,12.721772,14.870554,13.424563
1012,01005,16256,26.926675,28.968065,23.494000,10.857265,NaN,1.296720,14.776005,4.351032,NaN,10.237687,21.917558,13.834021,14.553288,13.008872
1013,01007,17178,13.390054,25.468740,32.623436,3.001344,NaN,NaN,12.837897,1.138476,NaN,3.879801,21.463564,16.420261,14.696877,11.998075
1014,01009,20774,14.788819,26.622537,21.418405,4.165103,1.158537,1.032048,14.575068,6.592957,NaN,0.972715,20.221689,13.581471,14.059999,13.057702


In [110]:
df.to_csv('SDoH_3years/Social_Economic_3years_2011.csv')